In [14]:
import numpy as np
import pandas as pd
import os
from libs.utils import *
from libs.tf_idf import *

In [6]:
DATASET_PATH = r"E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx"

data : pd.DataFrame = read_dataset(DATASET_PATH, "xlsx", header=0)

data.head()

15:21:03 root INFO: Đang đọc dữ liệu từ file E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx ...
		at Line 330 [read_dataset() in utils.py, utils]
15:21:04 root INFO: Đọc thành công! 
		at Line 338 [read_dataset() in utils.py, utils]


,repair_words,labels
0,"['tiếng', 'anh', 'thi', 'đầu_vào', 'là', 'khôn...",Neutral
1,"['đã', 'từng', 'là', 'niềm', 'mơ_ước', 'khi', ...",Negative
2,"['học_phí', 'hơi', 'cao']",Negative
3,"['̀', '̀', '̣', '̀', '̀', '̉', '̣', '̂', 'đ', ...",Neutral
4,"['thiết_nghĩ', 'đề', 'thi', 'utc', 'nên', 'đổi...",Positive


In [13]:
labels : list[str] = data["labels"].unique().tolist()
labels.sort(reverse=True)
labels

['Positive', 'Neutral', 'Negative']

In [18]:
import string
import re

string_pattele = string.punctuation + '\n'
string_pattele


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_punctuation(text, except_char: str = None):
    try:
        result_text = "".join(
            char for char in text if char not in string_pattele or (except_char and char == except_char))
        result_text = remove_emoji(result_text)
    except TypeError:
        result_text = None
    return result_text

def filter_none_text_id(documents : pd.Series | list[str]) -> list[int]:
    need_filter : list[str] = [text for text in documents]
    none_text_index : list[int] = []
    for i, text in enumerate(need_filter):
        if (text is None):
            print(str(i) + ": " + str(text))
            none_text_index.append(i)
    
    return none_text_index

In [20]:
data["Tokens"] = data["repair_words"].apply(lambda text: [remove_punctuation(x, '_').split()[0] for x in text.split(',')])

In [29]:
corpus: list[list[str]] = [doc for doc in data["Tokens"]]
labels_corpus: list[str] = data["labels"].to_list()

In [38]:
vocabulary = features_extraction(corpus)
vocabulary = list(vocabulary)
vocabulary.sort()

# 1. Xét n-gram = 1:

Tạo phân phối tần suất:

In [112]:
positive_frequency : dict[str, int] = {}
for token in vocabulary:
    positive_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Positive"].values:
    for token in sen:
        positive_frequency[token] += 1

negative_frequency : dict[str, int] = {}
for token in vocabulary:
    negative_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Negative"].values:
    for token in sen:
        negative_frequency[token] += 1

neutral_frequency : dict[str, int] = {}
for token in vocabulary:
    neutral_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Neutral"].values:
    for token in sen:
        neutral_frequency[token] += 1


In [113]:
positive_frequency = dict(sorted(positive_frequency.items(), key=lambda x:x[1], reverse=True))
negative_frequency = dict(sorted(negative_frequency.items(), key=lambda x:x[1], reverse=True))
neutral_frequency = dict(sorted(neutral_frequency.items(), key=lambda x:x[1], reverse=True))

Xét độ tạp chất của từng từ trong từ điển:

In [114]:
def gini_score(n_class_samples : list[int]) -> float:
    gini : float = 0
    n_samples : int = 0
    for n in n_class_samples:
        n_samples += n
    for n in n_class_samples:
        gini += (n / n_samples) ** 2
    
    return 1 - gini

In [115]:
gini_score_vocabulary : dict[str, float] = {}
for token in vocabulary:
    gini_score_vocabulary[token] = gini_score([
        positive_frequency[token],
        negative_frequency[token],
        neutral_frequency[token]
    ])

gini_score_vocabulary = dict(sorted(gini_score_vocabulary.items(), key=lambda x:x[1]))


In [116]:
THRESHOLD = 1
MIN_SAMPLES = 3

features_vocabulary : dict[str, str] = {} 

for key in gini_score_vocabulary:
    total_samples = positive_frequency[key] + negative_frequency[key] + neutral_frequency[key]
    if gini_score_vocabulary.get(key) <= THRESHOLD and total_samples >= MIN_SAMPLES:
        feq_max = max([
            positive_frequency[key],
            negative_frequency[key],
            neutral_frequency[key],
        ])
        if feq_max == positive_frequency[key]:
            features_vocabulary[key] = "Positive"
        elif feq_max == negative_frequency[key]:
            features_vocabulary[key] = "Negative"
        else:
            features_vocabulary[key] = "Neutral"


In [117]:
for key in features_vocabulary:
    if features_vocabulary.get(key) == "Negative":
        print(key, end=", ")

bcs, bán_nước, bẩn, bốt, bớt, bức_xúc, chiếm, cmn, cư_xử, cất, cắt, cốp, deadline, dong, dí, giang_hồ, giáo_vụ, grab, gà, gậy, hình_như, khâu, lão, lùa, lũ, may_ra, ngoại_hình, ngón, nhan_sắc, nón, nọ, phản_ánh, quát, soát, sập, thiếu_thốn, trườg, tệ, tổn_thương, web, xước, áo_mưa, ó, đơn_phương, đực, ốm, 1k, dắt, chửi, chặn, im, kêu, tắc, đếm, 2k, già, gọn, kẹt, phi, phụ, yếu, đổ, nhà_xe, 100k, 60, 9tr, chật, chốn, gắt, nhân_phẩm, thừa, vô_duyên, đắt, bà, vụ, 20k, 40, bách_khoa, chương, chịu_đựng, chở, cấm, dừng, hậu, học_kỳ, hổ, khó_chịu, làm_ăn, phóng, sửa, thi_công, tụi, xe_đạp, giải_quyết, leo, kém, cháu, a7, chú, mệt, xe, 54, 70, 90, a6, bố_mẹ, bộ_phận, canh, cs1, cuốn, cán_sự, cổ, cửa, dương, fes, gom, huỷ, hài_lòng, hành_chính, khổ, kéo, liên_thông, lì_xì, lườm, lịch_sự, lỗi, mơ_ước, mạnh_dạn, nuốt, nếu_như, oi, rbc, rác, rưỡi, suy, sáng_rực, sóng, sư_phạm, tay_lái, thậm_chí, thế_thì, trách, trốn, trộm, tát, tường, tỏ, từ_bỏ, tự_dưng, u, vch, vàng, vắng, vỏ, vỡ, vững, ám_ảnh, đ

# 2. Xét n-gram = 2:

In [ ]:
positive_frequency : dict[str, int] = {}
for token in vocabulary:
    positive_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Positive"].values:
    for token in sen:
        positive_frequency[token] += 1

negative_frequency : dict[str, int] = {}
for token in vocabulary:
    negative_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Negative"].values:
    for token in sen:
        negative_frequency[token] += 1

neutral_frequency : dict[str, int] = {}
for token in vocabulary:
    neutral_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Neutral"].values:
    for token in sen:
        neutral_frequency[token] += 1
